In [1]:
import clip
import torch 
import numpy as np
import os
import glob
from pathlib import Path
from PIL import Image

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [01:45<00:00, 3.36MiB/s]


In [6]:
from glob import glob

In [12]:
image_path = Path("../images")
images = glob(str(image_path / "*"))

In [13]:
images

['..\\images\\Blue and Yellow Personal Profile LinkedIn Background Banner.png',
 '..\\images\\Designer - Copy.png',
 '..\\images\\image1-139 - Copy.png',
 '..\\images\\IMG_20240303_232651 - Copy.jpg',
 '..\\images\\light2 - Copy (2).jpg',
 '..\\images\\light3 - Copy.jpg',
 '..\\images\\mockup copy 1.jpg',
 '..\\images\\moving light 2.jpeg',
 '..\\images\\moving light 3.jpeg',
 '..\\images\\Purple Internship Job Hiring Facebook Post.png',
 '..\\images\\WhatsApp Image 2024-01-11 at 19.35.39_496aed43 - Copy.jpg',
 '..\\images\\WhatsApp Image 2024-03-04 at 08.51.06_a0b11b3f - Copy.jpg',
 '..\\images\\WhatsApp Image 2024-03-17 at 11.14.13_2b120714.jpg',
 '..\\images\\WhatsApp Image 2024-04-04 at 23.36.40_e17d9888.jpg']

In [24]:
image_embedding = []
for i in images:
    image = preprocess(Image.open(i)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image).cpu().numpy().tolist()
    image_embedding.append(image_features)

In [22]:
import chromadb

In [44]:
chroma_client = chromadb.Client()
image_collection = chroma_client.create_collection("image" , metadata={"hnsw:space": "cosine"})

In [43]:
chroma_client.delete_collection("image")

In [30]:
ids_index = 1

In [45]:
for i in images:
    image = preprocess(Image.open(i)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image).cpu().numpy().tolist()
    image_embedding.append(image_features)
    ids_index += 1
    image_collection.add(ids= str(ids_index + 1) ,embeddings=image_features, metadatas={"path": i , "name": os.path.basename(i)})

In [67]:
text_input = input("Enter text: ")
text_embedding = clip.tokenize(text_input).to(device)
text_features = model.encode_text(text_embedding).detach().cpu().numpy()
result = image_collection.query(text_features, n_results=2)
for i in result['metadatas'][0]:
    print(i['name'])
    print(i['path'])
    img = Image.open(i['path'])
    img.show()

WhatsApp Image 2024-03-17 at 11.14.13_2b120714.jpg
..\images\WhatsApp Image 2024-03-17 at 11.14.13_2b120714.jpg
WhatsApp Image 2024-01-11 at 19.35.39_496aed43 - Copy.jpg
..\images\WhatsApp Image 2024-01-11 at 19.35.39_496aed43 - Copy.jpg
